In [5]:
def get_performance(strat):
    returns_analysis = strat.analyzers.getbyname('returns').get_analysis()
    anualreturns_analysis = strat.analyzers.getbyname('annualreturn').get_analysis()
    drawdown_analysis = strat.analyzers.getbyname('drawdown').get_analysis()
    sharpe_analysis = strat.analyzers.getbyname('sharperatio').get_analysis()
    trade_analysis = strat.analyzers.getbyname('tradeanalyzer').get_analysis()
    timedrawdown_analysis = strat.analyzers.getbyname('timedrawdown').get_analysis()
    
    columns = [
        'total return',
        'average return',
        'normalized return',
        'max drawdown',
        'money_drawdown',
        'time_drawdown',
        'sharpe_ratio',
        'total_trades',
        'total_won',
        'won _avg',
        'total_lost',
        'avg_lost',
        'net_pnl_trade',
    ]

    values = [
        returns_analysis['rtot'],
        returns_analysis['ravg'],
        returns_analysis['rnorm100'],
        drawdown_analysis['max']['drawdown'],
        drawdown_analysis['max']['moneydown'],
        timedrawdown_analysis['maxdrawdownperiod'],
        sharpe_analysis['sharperatio'],
        trade_analysis['total']['total'],
        trade_analysis['won']['total'],
        trade_analysis['won']['pnl']['average'],
        trade_analysis['lost']['total'],
        trade_analysis['lost']['pnl']['average'],
        trade_analysis['pnl']['net']['average'],
    ]


    performance = pd.DataFrame({'metric':columns, 'value':values})
    performance = performance.set_index('metric')


    analysis = strat.analyzers.getbyname('transactions').get_analysis()
    rows = []
    # Iterar sobre el OrderedDict y extraer los datos
    for date, transactions in analysis.items():
        for transaction in transactions:
            # Desempaquetar los valores de cada transacción
            amount, price, sid, symbol, value = transaction
            # Crear una fila con la fecha y los valores
            rows.append([date, amount, price, sid, symbol, value])

    # Crear un DataFrame con las filas y las columnas especificadas
    trades = pd.DataFrame(rows, columns=['date', 'amount', 'price', 'sid', 'symbol', 'value'])

    # Mostrar el DataFrame

    return performance, trades

In [7]:
import itertools
import yfinance as yf
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz

tickers = [
    # # Stocks extrangeros
    'NVDAm',
    'AVGOm',
    'METAm',
    'AMDm',
    'TSLAm',
    'MSFTm',
    'GOOGLm', 
    'NKEm', 

    # # Indices
    # 'US500m',
    # '^DJI',
    # '^IXIC',
    # '^RUT',


    # # Criptos 
    # 'BTC-USD',
    'ETHUSDm', 
    'LTCUSDm', 
    
    # # # Comodities
    # # 'GC=F', # Gold
    # 'USOILm', # Crude oil
    # 'UKOILm', # Crude oil
    # # 'SI=F', # Silver
    # # 'HG=F', # Copper
    # 'XNGUSDm', # Natural Gas
]

intervals = [
    mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_M30,
    # mt5.TIMEFRAME_M15,
]

parameter_combinations = list(itertools.product(
    tickers, intervals
))

symbols = {}

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
utc_from = datetime(2021, 1, 1, tzinfo=timezone)
utc_to = datetime(2024, 1, 1, tzinfo=timezone)

for ticker, interval in parameter_combinations:
    # Obtener las tasas históricas
    print(ticker)
    rates = mt5.copy_rates_range(ticker, interval, utc_from, utc_to)
    
    # Crear DataFrame con las tasas
    df = pd.DataFrame(rates)
    
    # Convertir el tiempo de segundos a formato datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')


    # Renombrar columnas para el ticker principal
    df = df.rename(columns={
        'time': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'tick_volume': 'Volume'
    }).set_index('Date')


    if ticker not in symbols.keys():
        symbols[ticker] = {}
        symbols[ticker][interval] = {}

    symbols[ticker][interval] = df


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

# Mostrar el DataFrame final

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
NVDAm
AVGOm
METAm
AMDm
TSLAm
MSFTm
GOOGLm
NKEm
ETHUSDm
LTCUSDm


True

In [3]:
from datetime import datetime
import backtrader as bt

class EndOfMonth(bt.Strategy):
    params = dict(risk=50)  # Porcentaje del capital a invertir en cada operación

    def __init__(self):
        self.last_buy_date = None

    def next(self):
        actual_date = self.datas[0].datetime.datetime(0)

        for data in self.datas:
            position_size = self.getposition(data).size

            if position_size:  # Si hay una posición abierta
                time_in_position = (actual_date - self.last_buy_date).days

                if time_in_position >= 5:
                    self.close(data=data)

            else:  # Si no hay posición abierta
                if actual_date.day == 25:  # Solo intenta comprar el día 25
                    cash_to_invest = self.broker.getvalue() * self.p.risk / 100
                    price = data.close[0]
                    
                    size = int(cash_to_invest / price)
                    
                    if size > 0:  # Asegúrate de que el tamaño sea mayor que cero
                        self.buy(size=size, data=data)
                        self.last_buy_date = actual_date  # Actualiza la fecha de la última compra


cerebro = bt.Cerebro()

for ticker in symbols:
    data_feed = bt.feeds.PandasData(dataname=symbols[ticker][16408])
    cerebro.adddata(data_feed, name=ticker)
    
cerebro.addstrategy(EndOfMonth)

cerebro.broker.setcash(5000)
cerebro.broker.setcommission(
    leverage=100,
    commission=7e-4,
)

cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.AnnualReturn)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TimeDrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Transactions)

print(f'start cash: {cerebro.broker.cash}')

results = cerebro.run()
strat = results[0]

cerebro.plot(figsize=(18, 10), iplot=False)

print(f'final cash: {cerebro.broker.cash}')
performance, transactions = get_performance(strat)

SyntaxError: invalid syntax (__init__.py, line 239)

In [4]:
pip uninstall backtrader

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
transactions